### Large Data Set Tutorial

When working with large data sets stored on your hard drive, there may be constraints associated with memory overhead preventing data encoding to a full set in a single pass. One solution is to populate a postprocess_dict on a subset of the data and then use that as a key to encode the remainder of the data on a consistent basis in chunks.

In this tutorial we demonstrate such practice, based on assumption that the full train or test data sets are available to pandas.read_csv, and encoding of the full data set is applied in chunks which are then downloaded to a seperate csv file with pandas.to_csv.

In [1]:
import pandas as pd

from Automunge import *
am = AutoMunge()

In [2]:
#these are the data sets we'll upload to encode
train_filename = 'train.csv'
test_filename = 'test.csv'

#these are filenames of the encoded data sets we'll download in chunks
encoded_train_filename  = 'train_encoded.csv'
encoded_test_filename = 'test_encoded.csv'

### 1) Populate a postprocess_dict using a subset of the train data.

This example is being performed on a small daa set for simplicity, the concepts extend equally well to large data sets stored on a local hard drive.

First we'll upload a subset of the training data to populate a postprocess_dict, which may then serve as a key for encoding additional train or test data on a consistent basis.

In [3]:
#generally we want an extra few zeros here, this is just for this example on titanic dataset
populate_ppd_rowcount = 500

#load the subset used to populate the postprocess_dict
df_train = pd.read_csv(train_filename, nrows=populate_ppd_rowcount)

train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict = \
am.automunge(df_train, 
             labels_column = 'Survived',
             shuffletrain=False,
            )

_______________
Begin Automunge



/Users/nicholasteague/opt/anaconda3/lib/python3.8/site-packages/Automunge/Automunger.py:34590: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[activation_column] = 0


______

versioning serial stamp:
_8.12_780381958165_2022-02-04T11:00:52.057960

Automunge returned train column set: 
['PassengerId_nmbr', 'Pclass_nmbr', 'Sex_bnry', 'Age_nmbr', 'SibSp_nmbr', 'Parch_nmbr', 'Fare_nmbr', 'PassengerId_NArw', 'Pclass_NArw', 'Name_NArw', 'Name_hash_0', 'Name_hash_1', 'Name_hash_2', 'Name_hash_3', 'Name_hash_4', 'Name_hash_5', 'Name_hash_6', 'Name_hash_7', 'Name_hash_8', 'Name_hash_9', 'Name_hash_10', 'Name_hash_11', 'Name_hash_12', 'Name_hash_13', 'Sex_NArw', 'Age_NArw', 'SibSp_NArw', 'Parch_NArw', 'Ticket_NArw', 'Ticket_hash_0', 'Ticket_hash_1', 'Ticket_hash_2', 'Fare_NArw', 'Cabin_NArw', 'Cabin_1010_0', 'Cabin_1010_1', 'Cabin_1010_2', 'Cabin_1010_3', 'Cabin_1010_4', 'Cabin_1010_5', 'Cabin_1010_6', 'Embarked_NArw', 'Embarked_1010_0', 'Embarked_1010_1']

Automunge returned ID column set: 
['Automunge_index']

Automunge returned label column set: 
['Survived_lbbn']

_______________
Automunge Complete



In [4]:
train.head()

,PassengerId_nmbr,Pclass_nmbr,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,PassengerId_NArw,Pclass_NArw,Name_NArw,...,Cabin_1010_0,Cabin_1010_1,Cabin_1010_2,Cabin_1010_3,Cabin_1010_4,Cabin_1010_5,Cabin_1010_6,Embarked_NArw,Embarked_1010_0,Embarked_1010_1
0,-1.726861,0.814301,1,-0.493571,0.369224,-0.487012,-0.515321,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,-1.719939,-1.602022,0,0.602527,0.369224,-0.487012,0.829748,0,0,0,...,0,1,1,0,0,0,0,0,0,1
2,-1.713018,0.814301,0,-0.219547,-0.497499,-0.487012,-0.501142,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,-1.706097,-1.602022,0,0.397009,0.369224,-0.487012,0.447794,0,0,0,...,0,0,1,1,1,1,1,0,1,1
4,-1.699176,0.814301,1,0.397009,-0.497499,-0.487012,-0.498516,0,0,0,...,0,0,0,0,0,0,0,0,1,1


### 2) Prepare the rest of the training set

In [5]:

#we'll save the automunge returned headers to initalize the dataframe
#and then prepare sequential chunks to append to this csv
#note that the oriignal index is in the ID set if we need it
train = pd.concat([train, labels], axis=1)

train[:0].to_csv(encoded_train_filename, header=True, index=False)

#we can then prepare the full training data set in chunks
#chunk size is the size of paritions, you may want to add a zero or few
#depending on how much ram you have available

#note that the returnedsets option results in postmunge(.) only returning a single dataframe
#with labels concatinatted as final coumn(s) onto test features

for chunk in pd.read_csv(train_filename, chunksize=100, header=0):
  
  test = \
  am.postmunge(postprocess_dict, 
               chunk,
               printstatus=False,
               returnedsets='test_labels'
              )
  
  test.to_csv(encoded_train_filename, mode='a', header=False, index=False)

** Note that postmunge(.) latency is expected to trend better with increasing chunk sizes.

In [7]:
#we can inspect the downloaded result to confirm

df_inspect = pd.read_csv(encoded_train_filename)

print(df_inspect.shape)
df_inspect.head()

(891, 45)


,PassengerId_nmbr,Pclass_nmbr,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,PassengerId_NArw,Pclass_NArw,Name_NArw,...,Cabin_1010_1,Cabin_1010_2,Cabin_1010_3,Cabin_1010_4,Cabin_1010_5,Cabin_1010_6,Embarked_NArw,Embarked_1010_0,Embarked_1010_1,Survived_lbbn
0,-1.726861,0.814301,1,-0.493571,0.369224,-0.487012,-0.515321,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1,-1.719939,-1.602022,0,0.602527,0.369224,-0.487012,0.829748,0,0,0,...,1,1,0,0,0,0,0,0,1,0
2,-1.713018,0.814301,0,-0.219547,-0.497499,-0.487012,-0.501142,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,-1.706097,-1.602022,0,0.397009,0.369224,-0.487012,0.447794,0,0,0,...,0,1,1,1,1,1,0,1,1,0
4,-1.699176,0.814301,1,0.397009,-0.497499,-0.487012,-0.498516,0,0,0,...,0,0,0,0,0,0,0,1,1,1


### 3) Prepare test data for inference

In [8]:
#The primary difference of preparing the test data is that we won't have labels.

#we'll prepare an empty dataframe with column headers to initalize the csv
#and then prepare the test data in a similar fashion

#initialize headers for csv
finalcolumns_train = postprocess_dict['finalcolumns_train']

headers_df = pd.DataFrame( dict(zip(finalcolumns_train, [[]] * len(finalcolumns_train) )) )

#this prepares the csv to include headers
headers_df.to_csv(encoded_test_filename, header=True, index=False)

In [9]:
#and then prepare the encoded test dataframe
#note that the returnedsets=False just returns the features without labels
#obivously can add a few zero's to the chunksize
for chunk in pd.read_csv(test_filename, chunksize=100, header=0):
  
  test = \
  am.postmunge(postprocess_dict, 
               chunk,
               printstatus=False,
               returnedsets=False,
              )
  
  test.to_csv(encoded_test_filename, mode='a', header=False, index=False)

In [10]:
#we can inspect the downloaded result to confirm

df_inspect = pd.read_csv(encoded_test_filename)

print(df_inspect.shape)
df_inspect.head()

(418, 44)


,PassengerId_nmbr,Pclass_nmbr,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,PassengerId_NArw,Pclass_NArw,Name_NArw,...,Cabin_1010_0,Cabin_1010_1,Cabin_1010_2,Cabin_1010_3,Cabin_1010_4,Cabin_1010_5,Cabin_1010_6,Embarked_NArw,Embarked_1010_0,Embarked_1010_1
0,4.440004,0.814301,1,0.362756,-0.497499,-0.487012,-0.503154,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,4.446926,0.814301,0,1.219083,0.369224,-0.487012,-0.520572,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,4.453847,-0.393861,1,2.246675,-0.497499,-0.487012,-0.464119,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,4.460768,0.814301,1,-0.151041,-0.497499,-0.487012,-0.485650,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,4.467689,0.814301,0,-0.493571,0.369224,0.794599,-0.409504,0,0,0,...,0,0,0,0,0,0,0,0,1,1
